# Train SALLY

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

from madminer.sampling import SampleAugmenter
from madminer.sampling import multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'

## Forge instance

In [6]:
forge = MLForge(debug=False)

08:20  
08:20  ------------------------------------------------------------
08:20  |                                                          |
08:20  |  MadMiner v2018.10.12                                    |
08:20  |                                                          |
08:20  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
08:20  |                                                          |
08:20  ------------------------------------------------------------
08:20  


## Train SALLY on all observables

In [7]:
forge.train(
    method='sally',
    x_filename=sample_dir + 'train_local/x_train_local.npy',
    t_xz0_filename=sample_dir + 'train_local/t_xz_train_local.npy',
    n_epochs=20,
    batch_size=256,
    validation_split=0.5
)

forge.save(model_dir + 'sally_all')

08:20  Starting training
08:20    Method:                 sally
08:20    Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/x_train_local.npy
08:20                   t_xz (theta0) at  /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/t_xz_train_local.npy
08:20    Features:               all
08:20    Method:                 sally
08:20    Hidden layers:          (100, 100, 100)
08:20    Activation function:    tanh
08:20    Batch size:             256
08:20    Epochs:                 20
08:20    Learning rate:          0.001 initially, decaying to 0.0001
08:20    Validation split:       0.5
08:20    Early stopping:         True
08:20  Loading training data
08:20  Found 1000000 samples with 2 parameters and 27 observables
08:20  Creating model for method sally
08:20  Training model
08:21    Epoch 2: train loss 15.86 ([15.85742034]), validation loss 61.86 ([61.86236478])
08:21    Epoch 4: 

## Train just on  lepton pT, photon pT, MET

In [8]:
forge.train(
    method='sally',
    x_filename=sample_dir + 'train_local/x_train_local.npy',
    t_xz0_filename=sample_dir + 'train_local/t_xz_train_local.npy',
    features=[0,4,9],
    n_epochs=20,
    batch_size=256,
    validation_split=0.5
)

forge.save(model_dir + 'sally_pts')

08:26  Starting training
08:26    Method:                 sally
08:26    Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/x_train_local.npy
08:26                   t_xz (theta0) at  /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/t_xz_train_local.npy
08:26    Features:               [0, 4, 9]
08:26    Method:                 sally
08:26    Hidden layers:          (100, 100, 100)
08:26    Activation function:    tanh
08:26    Batch size:             256
08:26    Epochs:                 20
08:26    Learning rate:          0.001 initially, decaying to 0.0001
08:26    Validation split:       0.5
08:26    Early stopping:         True
08:26  Loading training data
08:26  Found 1000000 samples with 2 parameters and 27 observables
08:26  Only using 3 / 27 observables
08:26  Creating model for method sally
08:26  Training model
08:27    Epoch 2: train loss 21.72 ([21.71924807]), validation lo

## Train just on delta phis

In [9]:
forge.train(
    method='sally',
    x_filename=sample_dir + 'train_local/x_train_local.npy',
    t_xz0_filename=sample_dir + 'train_local/t_xz_train_local.npy',
    features=[21,22,25],
    n_epochs=20,
    batch_size=256,
    validation_split=0.5
)

forge.save(model_dir + 'sally_deltaphis')

08:33  Starting training
08:33    Method:                 sally
08:33    Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/x_train_local.npy
08:33                   t_xz (theta0) at  /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/t_xz_train_local.npy
08:33    Features:               [21, 22, 25]
08:33    Method:                 sally
08:33    Hidden layers:          (100, 100, 100)
08:33    Activation function:    tanh
08:33    Batch size:             256
08:33    Epochs:                 20
08:33    Learning rate:          0.001 initially, decaying to 0.0001
08:33    Validation split:       0.5
08:33    Early stopping:         True
08:33  Loading training data
08:33  Found 1000000 samples with 2 parameters and 27 observables
08:33  Only using 3 / 27 observables
08:33  Creating model for method sally
08:33  Training model
08:33    Epoch 2: train loss 55.29 ([55.29170362]), validation